# Scalable estimation of pure multi-qubit states

This notebook shows an example for the paper https://arxiv.org/pdf/2107.05691.pdf. This is a method that estimates a $n$-qubit pure state measuring only $mn+1$ separable bases or $m$ entangled bases plus the computational basis, with $m$ larger or equal than 2. We want to simulate the tomography of a system of $15$ qubits using $4n+1$ separable bases. 

First, we import the required packages and functions

In [7]:
import numpy as np
from core.tresbases_vec import bases_2_3, bases_separables_vec, tomography_vec
from core.utils import estado, estado_sep, fidelidad_vec, dot_prod_vec
import time

We fix an arbitrary state for a system of $15$ qubits

In [8]:
n_qubits = 15
dim = int(2**n_qubits)
psi_sistema = estado(dim, 1)
# this is the number of shots in our experiment.
nu_exp = 2**18

Now, we select the appropriate bases. In this case, they are $4n +1$ separable bases, that is, we do not need CNOT gates to implement them. 

In [9]:
# These are the parameters of the bases
v_a = np.array([1/np.sqrt(2), 1/np.sqrt(2), 1/np.sqrt(2), 1/np.sqrt(2)])
v_fase = np.array([0, np.pi/2, np.pi/3, np.pi/4])
v_b = np.sqrt(1 - v_a**2)
n_bases = v_b.shape[0]
# These are the bases
base_diag_vec, bases_sep_vec = bases_separables_vec(dim, v_a, v_b, v_fase)

We simulate the measurements in our system using nu_exp shots for each base.

In [10]:
start = time.time()
prob_sep_vec = np.zeros((dim, n_qubits, n_bases))
prob_diag_vec = fidelidad_vec(psi_sistema, base_diag_vec, nu_exp = nu_exp)
for k in range(n_bases):
    for j in range(n_qubits):
        prob_sep_vec[:, j, k] = fidelidad_vec(psi_sistema,
                                              bases_sep_vec[:, :, :, j, k],
                                              nu_exp=nu_exp)
end = time.time()
print("time_measurements", end - start)

time_measurements 0.4868800640106201


Once we have the histograms with the measurements, we use our method to obtain an estimated quantum state for the system.

In [12]:
start = time.time()
psi = tomography_vec(prob_diag_vec, prob_sep_vec, bases_sep_vec)
end = time.time()
print("time_reconstruction", end - start)
print("fidelity", np.abs(np.dot(psi_sistema.conj().T, psi))**2)

time_reconstruction 8.32308030128479
fidelity [[0.91996115]]


Since the method is efficient, we can reconstruct a state of $15$ qubits in less than $10$ seconds, with a fidelity grater than $0.9$. 